In [2]:
import lasagne
import theano
from theano import tensor as T
import sys
import numpy as np
#enable importing of notebooks
from nbfinder import NotebookFinder
sys.meta_path.append(NotebookFinder())
from detec_helper_fxns import get_best_box, get_detec_loss, get_iou, make_test_data, get_detec_acc

importing Jupyter notebook from detec_helper_fxns.ipynb
importing Jupyter notebook from print_n_plot.ipynb
importing Jupyter notebook from helper_fxns.ipynb


In [3]:
def build_det_network_layers(class_net, layers_to_remove,
                                  num_filters,
                                  num_fc_units,
                                  num_extra_conv, 
                                  nonlinearity,
                                  n_boxes,
                                  nclass,
                                  grid_size,
                                  w_init,
                                  dropout_p):
    #remove last two layers
    for i in range(layers_to_remove):
        class_net = class_net.input_layer
        
    #add new layers
    network = lasagne.layers.Conv2DLayer(class_net, num_filters=num_filters, 
                                     filter_size=(3,3),
                                     pad=1,
                                     nonlinearity=nonlinearity,
                                     W=w_init)
    
    
    
    for i in range(num_extra_conv):
        network = lasagne.layers.Conv2DLayer(network, num_filters=num_filters, 
                                             filter_size=(3,3),
                                             pad=1,
                                             nonlinearity=nonlinearity,
                                             W=w_init)


    
    network = lasagne.layers.DenseLayer(
                                lasagne.layers.dropout(network, p=dropout_p),
                                num_units=num_fc_units,
                                nonlinearity=lasagne.nonlinearities.rectify)
    
    network = lasagne.layers.DenseLayer(network, 
                                        num_units=(grid_size * grid_size) * (n_boxes* 5 + nclass),
                                        nonlinearity=lasagne.nonlinearities.linear)
    network = lasagne.layers.ReshapeLayer(network, shape=(grid_size, grid_size,(n_boxes* 5 + nclass)))
    
    return network 
    

In [4]:
def build_det_network(class_net,
                      learning_rate = 0.001,
                      momentum = 0.9,
                      layers_to_remove=3,
                      num_filters=512,
                      num_fc_units=1024,
                      num_extra_conv=1, 
                      nonlinearity=lasagne.nonlinearities.LeakyRectify(0.1),
                      n_boxes=2,
                      nclass=2,
                      grid_size=7,
                      w_init=lasagne.init.HeNormal(),
                      dropout_p=0.5,
                      lc = 5,
                      ln = 0.5,
                      load=False):
    
    input_var = T.tensor4('inputs')
    target_var = T.tensor4('targets') #is of shape (grid_size, grid_size,(n_boxes* 5 + nclass)
    print("Building model and compiling functions...")
    
    
    network = build_det_network_layers(class_net, layers_to_remove,
                                  num_filters,
                                  num_fc_units,
                                  num_extra_conv, 
                                  nonlinearity,
                                  n_boxes,
                                  nclass,
                                  grid_size,
                                  w_init,
                                  dropout_p)
    
    if load:
        pass
#         with np.load('model.npz') as f:
#             param_values = [f['arr_%d' % i] for i in range(len(f.files))]
#             lasagne.layers.set_all_param_values(network, param_values)

 
    prediction = lasagne.layers.get_output(network, deterministic=False)
    
    #returns symbolic expression for loss
    loss = get_detec_loss(prediction, target_var, lc, ln)

    params = lasagne.layers.get_all_params(network, trainable=True)
    updates = lasagne.updates.nesterov_momentum(loss, 
                                                params, 
                                                learning_rate=learning_rate, 
                                                momentum=momentum)

    test_prediction = lasagne.layers.get_output(network, deterministic=True)
    
    #get non-maximally suppressed bbox
    test_loss = get_detec_loss(test_prediction, target_var, lc, ln)


    pred_boxes = get_best_box(prediction)
    gt_boxes = get_best_box(target_var)
    # As a bonus, also create an expression for the classification accuracy:
    #we use avg IOU here?
    test_acc = get_detec_acc(test_prediction, target_var)

    # Compile a function performing a training step on a mini-batch (by giving
    # the updates dictionary) and returning the corresponding training loss:
    train_fn = theano.function([input_var, target_var], loss, updates=updates)


    # Compile a second function computing the validation loss and accuracy:
    val_fn = theano.function([input_var, target_var], [test_loss, test_acc])
    box_fn = theano.function([input_var, target_var], [pred_boxes, gt_boxes])

    return train_fn, val_fn, network, box_fn